<div class="align-center">
<a href="https://oumi.ai/"><img src="https://oumi.ai/docs/en/latest/_static/logo/header_logo.png" height="200"></a>

[![Documentation](https://img.shields.io/badge/Documentation-latest-blue.svg)](https://oumi.ai/docs/en/latest/index.html)
[![Discord](https://img.shields.io/discord/1286348126797430814?label=Discord)](https://discord.gg/oumi)
[![GitHub Repo stars](https://img.shields.io/github/stars/oumi-ai/oumi)](https://github.com/oumi-ai/oumi)
<a target="_blank" href="https://colab.research.google.com/github/oumi-ai/oumi/blob/main/notebooks/Oumi - A Tour.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>

👋 Welcome to Open Universal Machine Intelligence (Oumi)!

🚀 Oumi is a fully open-source platform that streamlines the entire lifecycle of foundation models - from [data preparation](https://oumi.ai/docs/en/latest/resources/datasets/datasets.html) and [training](https://oumi.ai/docs/en/latest/user_guides/train/train.html) to [evaluation](https://oumi.ai/docs/en/latest/user_guides/evaluate/evaluate.html) and [deployment](https://oumi.ai/docs/en/latest/user_guides/launch/launch.html). Whether you're developing on a laptop, launching large scale experiments on a cluster, or deploying models in production, Oumi provides the tools and workflows you need.

🤝 Make sure to join our [Discord community](https://discord.gg/oumi) to get help, share your experiences, and contribute to the project! If you are interested in joining one of the community's open-science efforts, check out our [open collaboration](https://oumi.ai/community) page.

⭐ If you like Oumi and you would like to support it, please give it a star on [GitHub](https://github.com/oumi-ai/oumi).

# A Tour of Oumi

This tutorial will give you a brief overview of Oumi's core functionality. We'll cover:

1. Training a model
1. Performing model inference
1. Evaluating a model against common benchmarks
1. Launching jobs
1. Customizing datasets and clouds

# 📋 Prerequisites

❗**NOTICE:** We recommend running this notebook on a GPU. If running on Google Colab, you can use the free T4 GPU runtime (Colab Menu: `Runtime` -> `Change runtime type`).

## Oumi Installation

First, let's install Oumi. You can find more detailed instructions [here](https://oumi.ai/docs/en/latest/get_started/installation.html).


In [1]:
%pip install oumi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 550.0/550.0 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 73.5 MB/s eta 0:00:

In [3]:
import os
from pathlib import Path

tutorial_dir = "tour_tutorial"

Path(tutorial_dir).mkdir(parents=True, exist_ok=True)
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Disable warnings from HF.

# ⚒️ Training a Model

Oumi supports training both custom and out-of-the-box models. Want to try out a model on HuggingFace? You can do that. Want to train your own custom Pytorch model? No problem.

## A Quick Demo

Let's try training a pre-existing model on HuggingFace. We'll use SmolLM2 135M as it's small and trains quickly.

Oumi uses [training configuration files](https://oumi.ai/docs/en/latest/api/oumi.core.configs.html#oumi.core.configs.TrainingConfig) to specify training parameters. We've already created a training config for SmolLM2 — let's give it a try!

In [4]:
yaml_content = f"""
model:
  model_name: "HuggingFaceTB/SmolLM2-135M-Instruct"
  torch_dtype_str: "bfloat16"
  trust_remote_code: True

data:
  train:
    datasets:
      - dataset_name: "yahma/alpaca-cleaned"
    target_col: "prompt"

training:
  trainer_type: "TRL_SFT"
  per_device_train_batch_size: 2
  max_steps: 10 # Quick "mini" training, for demo purposes only.
  run_name: "smollm2_135m_sft"
  output_dir: "{tutorial_dir}/output"
"""

with open(f"{tutorial_dir}/train.yaml", "w") as f:
    f.write(yaml_content)

In [5]:
from oumi.core.configs import TrainingConfig
from oumi.train import train

config = TrainingConfig.from_yaml(str(Path(tutorial_dir) / "train.yaml"))

train(config)

[2025-05-16 02:36:03,906][oumi][rank0][pid:1873][MainThread][INFO]][train.py:112] Creating training.output_dir: tour_tutorial/output...
[2025-05-16 02:36:03,907][oumi][rank0][pid:1873][MainThread][INFO]][train.py:114] Created training.output_dir absolute path: /content/tour_tutorial/output
[2025-05-16 02:36:03,909][oumi][rank0][pid:1873][MainThread][INFO]][train.py:112] Creating training.telemetry_dir: tour_tutorial/output/telemetry...
[2025-05-16 02:36:03,910][oumi][rank0][pid:1873][MainThread][INFO]][train.py:114] Created training.telemetry_dir absolute path: /content/tour_tutorial/output/telemetry
[2025-05-16 02:36:03,911][oumi][rank0][pid:1873][MainThread][INFO]][torch_utils.py:81] Torch version: 2.5.1+cu124. NumPy version: 1.26.4
[2025-05-16 02:36:03,912][oumi][rank0][pid:1873][MainThread][INFO]][torch_utils.py:89] CUDA version: 12.4 
[2025-05-16 02:36:03,935][oumi][rank0][pid:1873][MainThread][INFO]][torch_utils.py:92] CuDNN version: 90.1.0
[2025-05-16 02:36:04,162][oumi][rank0][

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

[2025-05-16 02:36:13,400][oumi][rank0][pid:1873][MainThread][INFO]][base_map_dataset.py:312] AlpacaDataset: features=dict_keys(['input_ids', 'attention_mask'])


Generating train split: 0 examples [00:00, ? examples/s]

[2025-05-16 02:37:08,242][oumi][rank0][pid:1873][MainThread][INFO]][base_map_dataset.py:376] Finished transforming dataset (AlpacaDataset)! Speed: 943.90 examples/sec. Examples: 51760. Duration: 54.8 sec. Transform workers: 1.
[2025-05-16 02:37:08,261][oumi][rank0][pid:1873][MainThread][INFO]][train.py:329] Skipping dataset preparation for TRL_SFT trainer since the dataset is already processed.
[2025-05-16 02:37:08,846][oumi][rank0][pid:1873][MainThread][INFO]][torch_profiler_utils.py:164] PROF: Torch Profiler disabled!
[2025-05-16 02:37:08,872][oumi][rank0][pid:1873][MainThread][INFO]][training.py:62] SFTConfig(output_dir='tour_tutorial/output',
          overwrite_output_dir=False,
          do_train=False,
          do_eval=False,
          do_predict=False,
          eval_strategy=<IntervalStrategy.NO: 'no'>,
          prediction_loss_only=False,
          per_device_train_batch_size=2,
          per_device_eval_batch_size=8,
          per_gpu_train_batch_size=None,
          per_g

Step,Training Loss


[2025-05-16 02:37:17,217][oumi][rank0][pid:1873][MainThread][INFO]][train.py:428] Training is Complete.
[2025-05-16 02:37:17,222][oumi][rank0][pid:1873][MainThread][INFO]][device_utils.py:297] GPU Metrics After Training: GPU runtime info: NVidiaGpuRuntimeInfo(device_index=0, device_count=1, used_memory_mb=2997.0, temperature=47, fan_speed=None, fan_speeds=None, power_usage_watts=27.663, power_limit_watts=70.0, gpu_utilization=0, memory_utilization=0, performance_state=0, clock_speed_graphics=585, clock_speed_sm=585, clock_speed_memory=5000).
[2025-05-16 02:37:17,223][oumi][rank0][pid:1873][MainThread][INFO]][torch_utils.py:136] Peak GPU memory usage: 2.06 GB
[2025-05-16 02:37:17,224][oumi][rank0][pid:1873][MainThread][INFO]][train.py:435] Saving final state...
[2025-05-16 02:37:17,225][oumi][rank0][pid:1873][MainThread][INFO]][train.py:440] Saving final model...
[2025-05-16 02:37:18,104][oumi][rank0][pid:1873][MainThread][INFO]][hf_trainer.py:116] Model has been saved at tour_tutorial/

Congratulations, you've trained your first model using Oumi!

You can also train your own custom Pytorch model. We cover that in depth in our [Finetuning Tutorial](https://github.com/oumi-ai/oumi/blob/main/notebooks/Oumi%20-%20Finetuning%20Tutorial.ipynb).

# 🧠 Model Inference

Now that you've trained a model, let's run inference.

In [6]:
yaml_content = f"""
model:
  model_name: "{tutorial_dir}/output"
  torch_dtype_str: "bfloat16"

generation:
  max_new_tokens: 128
  batch_size: 1
"""

with open(f"{tutorial_dir}/infer.yaml", "w") as f:
    f.write(yaml_content)

In [7]:
from oumi.core.configs import InferenceConfig
from oumi.infer import infer

config = InferenceConfig.from_yaml(str(Path(tutorial_dir) / "infer.yaml"))

input_text = (
    "Remember that we didn't train for long, so the results might not be great."
)

results = infer(config=config, inputs=[input_text])

print(results[0])

[2025-05-16 02:37:54,233][oumi][rank0][pid:1873][MainThread][WARNING]][infer.py:33] No inference engine specified. Using the default 'native' engine.
[2025-05-16 02:37:54,236][oumi][rank0][pid:1873][MainThread][INFO]][models.py:208] Building model using device_map: auto (DeviceRankInfo(world_size=1, rank=0, local_world_size=1, local_rank=0))...
[2025-05-16 02:37:54,239][oumi][rank0][pid:1873][MainThread][INFO]][models.py:276] Using model class: <class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'> to instantiate model.
[2025-05-16 02:37:54,859][oumi][rank0][pid:1873][MainThread][INFO]][models.py:482] Using the model's built-in chat template for model 'tour_tutorial/output'.
[2025-05-16 02:37:54,863][oumi][rank0][pid:1873][MainThread][INFO]][native_text_inference_engine.py:141] Setting EOS token id to `2`


`generation_config` default values have been modified to match model-specific defaults: {'pad_token_id': 2, 'bos_token_id': 1}. If this is not desired, please set these values explicitly.


conversation_id=None messages=[USER: Remember that we didn't train for long, so the results might not be great., ASSISTANT: I'm sorry for the inconvenience, but as a chatbot, I don't have the ability to access or process data from external sources. I'm designed to provide information and guidance based on your input, not to collect or store any data. I recommend checking the official website or platform for the most accurate and up-to-date information.] metadata={}


We can also run inference using the pretrained model by slightly tweaking our config:

In [8]:
base_model_config = InferenceConfig.from_yaml(str(Path(tutorial_dir) / "infer.yaml"))
base_model_config.model.model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

input_text = "Input for the pretrained model: What is your name? "

results = infer(config=base_model_config, inputs=[input_text])

print(results[0])

[2025-05-16 02:38:23,841][oumi][rank0][pid:1873][MainThread][WARNING]][infer.py:33] No inference engine specified. Using the default 'native' engine.
[2025-05-16 02:38:23,843][oumi][rank0][pid:1873][MainThread][INFO]][models.py:208] Building model using device_map: auto (DeviceRankInfo(world_size=1, rank=0, local_world_size=1, local_rank=0))...
[2025-05-16 02:38:24,000][oumi][rank0][pid:1873][MainThread][INFO]][models.py:276] Using model class: <class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'> to instantiate model.
[2025-05-16 02:38:24,609][oumi][rank0][pid:1873][MainThread][INFO]][models.py:482] Using the model's built-in chat template for model 'HuggingFaceTB/SmolLM2-135M-Instruct'.
[2025-05-16 02:38:24,612][oumi][rank0][pid:1873][MainThread][INFO]][native_text_inference_engine.py:141] Setting EOS token id to `2`
conversation_id=None messages=[USER: Input for the pretrained model: What is your name? , ASSISTANT: My name is Alex Chen. I'm a data scientist and AI as

# 📊 Evaluating a Model against Common Benchmarks

You can use Oumi to evaluate pretrained and tuned models against standard benchmarks. For example, let's evaluate our tuned model against `Hellaswag`:

In [9]:
yaml_content = f"""
model:
  model_name: "{tutorial_dir}/output"
  torch_dtype_str: "bfloat16"

tasks:
  - evaluation_backend: lm_harness
    task_name: mmlu_college_computer_science

generation:
  batch_size: null # This will let LM HARNESS find the maximum possible batch size.
output_dir: "{tutorial_dir}/output/evaluation"
"""

with open(f"{tutorial_dir}/eval.yaml", "w") as f:
    f.write(yaml_content)

In [10]:
from oumi.core.configs import EvaluationConfig
from oumi.evaluate import evaluate

eval_config = EvaluationConfig.from_yaml(str(Path(tutorial_dir) / "eval.yaml"))

# Uncomment the following line to run evals against the V1 HuggingFace Leaderboard.
# This may take a while.
# eval_config.data.datasets[0].dataset_name = "huggingface_leaderboard_v1"

evaluate(eval_config)

[2025-05-16 02:38:49,455][oumi][rank0][pid:1873][MainThread][WARNING]][lm_harness.py:311] Since you have GPU support, it is highly recommended that you set the `inference_engine` to `VLLM`, instead of the `NATIVE`, for faster evaluation.


README.md:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

mmlu_no_train.py:   0%|          | 0.00/5.86k [00:00<?, ?B/s]

data.tar:   0%|          | 0.00/166M [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

...

[2025-05-16 02:39:04,871][oumi][rank0][pid:1873][MainThread][INFO]][lm_harness.py:331] 	LM Harness `task_params`:
LMHarnessTaskParams(evaluation_backend='lm_harness',
                    task_name='mmlu_college_computer_science',
                    num_samples=None,
                    log_samples=False,
                    eval_kwargs={},
                    evaluation_platform='',
                    num_fewshot=None)
[2025-05-16 02:39:04,873][oumi][rank0][pid:1873][MainThread][INFO]][lm_harness.py:332] 	LM Harness `task_dict`:
{'mmlu_college_computer_science': ConfigurableTask(task_name=mmlu_college_computer_science,output_type=multiple_choice,num_fewshot=None,num_samples=100)}
[2025-05-16 02:39:04,873][oumi][rank0][pid:1873][MainThread][INFO]][lm_harness.py:344] 	LM Harness `model_params`:
{'batch_size': 1,
 'device': 'cuda:0',
 'device_map': 'auto',
 'dtype': torch.bfloat16,
 'max_batch_size': None,
 'max_length': None,
 'parallelize': False,
 'pretrained': 'tour_tutorial/output'

Running loglikelihood requests: 100%|██████████| 400/400 [00:05<00:00, 79.67it/s]

[2025-05-16 02:39:11,274][oumi][rank0][pid:1873][MainThread][INFO]][lm_harness.py:366] mmlu_college_computer_science's metric dict is {'acc,none': 0.25,
 'acc_stderr,none': 0.04351941398892446,
 'alias': 'college_computer_science'}


[2025-05-16 02:39:14,008][oumi][rank0][pid:1873][MainThread][WARNING]][serialization_utils.py:47] Non-serializable value `LMHarnessTaskParams(evaluation_backend='lm_harness', task_name='mmlu_college_computer_science', num_samples=None, log_samples=False, eval_kwargs={}, evaluation_platform='', num_fewshot=None)` of type `<class 'oumi.core.configs.params.evaluation_params.LMHarnessTaskParams'>`.
[2025-05-16 02:39:14,009][oumi][rank0][pid:1873][MainThread][WARNING]][serialization_utils.py:47] Non-serializable value `ConfigurableTask(task_name=mmlu_college_computer_science,output_type=multiple_choice,num_fewshot=None,num_samples=100)` of type `<class 'lm_eval.api.task.ConfigurableTask'>`.


[{'results': {'mmlu_college_computer_science': {'alias': 'college_computer_science',
    'acc,none': 0.25,
    'acc_stderr,none': 0.04351941398892446}}}]

# ☁️ Launching Jobs

Oftentimes you'll need to run various tasks (training, evaluation, etc.) on remote hardware that's better suited for the task. Oumi can handle this for you by launching jobs on various compute clusters. For more information about running jobs, see our [Running Jobs Remotely tutorial](https://github.com/oumi-ai/oumi/blob/main/notebooks/Oumi%20-%20Running%20Jobs%20Remotely.ipynb). For running jobs on custom clusters, see our [Launching Jobs on Custom Clusters tutorial](https://github.com/oumi-ai/oumi/blob/main/notebooks/Oumi%20-%20Launching%20Jobs%20on%20Custom%20Clusters.ipynb).


Today, Oumi supports running jobs on several cloud provider platforms.

For the latest list, we can run the `which_clouds` method:

In [11]:
import oumi.launcher as launcher

print("Supported Clouds in Oumi:")
for cloud in launcher.which_clouds():
    print(cloud)

Supported Clouds in Oumi:
local
polaris
runpod
gcp
lambda
aws
azure
slurm


Let's run a simple "Hello World" job locally to demonstrate how to use the Oumi job launcher. This job will echo `Hello World`, then run the same training job executed above. Running this job on a cloud provider like GCP simply involves changing the `cloud` field.

In [12]:
yaml_content = f"""
name: hello-world
resources:
  cloud: local

working_dir: .

envs:
  TEST_ENV_VARIABLE: '"Hello, World!"'
  OUMI_LOGGING_DIR: "{tutorial_dir}/logs"

run: |
  echo "$TEST_ENV_VARIABLE"
  oumi train -c {tutorial_dir}/train.yaml
"""

with open(f"{tutorial_dir}/job.yaml", "w") as f:
    f.write(yaml_content)

In [13]:
import time

job_config = launcher.JobConfig.from_yaml(str(Path(tutorial_dir) / "job.yaml"))
cluster, job_status = launcher.up(job_config, cluster_name=None)

while job_status and not job_status.done:
    print("Job is running...")
    time.sleep(15)
    job_status = cluster.get_job(job_status.id)
print("Job is done!")

Job is running...
Job is running...
Job is running...
Job is done!


The job created logs under our tutorial directory. Let's take a look at the directory:

In [14]:
logs_dir = f"{tutorial_dir}/logs"
Path(logs_dir).iterdir()

<generator object Path.iterdir at 0x7baf8c1bfe60>

Now let's parse the logfiles.

In [15]:
for log_file in Path(logs_dir).iterdir():
    print(f"Log file: {log_file}")
    with open(log_file) as f:
        print(f.read())

Log file: tour_tutorial/logs/2025_05_16_02_40_07_905_0.stdout
"Hello, World!"

   ____  _    _ __  __ _____
  / __ \| |  | |  \/  |_   _|
 | |  | | |  | | \  / | | |
 | |  | | |  | | |\/| | | |
 | |__| | |__| | |  | |_| |_
  \____/ \____/|_|  |_|_____|
[2025-05-16 02:40:17,364][oumi][rank0][pid:3519][MainThread][INFO]][distributed.py:561] Setting random seed to 42 on rank 0.
[2025-05-16 02:40:22,563][oumi][rank0][pid:3519][MainThread][INFO]][torch_utils.py:81] Torch version: 2.5.1+cu124. NumPy version: 1.26.4
[2025-05-16 02:40:22,563][oumi][rank0][pid:3519][MainThread][INFO]][torch_utils.py:89] CUDA version: 12.4 
[2025-05-16 02:40:22,564][oumi][rank0][pid:3519][MainThread][INFO]][torch_utils.py:92] CuDNN version: 90.1.0
[2025-05-16 02:40:22,565][oumi][rank0][pid:3519][MainThread][INFO]][torch_utils.py:125] CPU cores: 2 CUDA devices: 1
device(0)='Tesla T4' Capability: (7, 5) Memory: [Total: 14.74GiB Free: 11.67GiB Allocated: 0.0GiB Cached: 0.0GiB]
[2025-05-16 02:40:22,567][oumi][rank0]

# ⚙️ Customizing Datasets and Clusters

Oumi offers rich customization that allows users to build custom solutions on top of our existing building blocks. Several of Oumi's primary resources (Datasets, Clouds, etc.) leverage the Oumi Registry when invoked.

This registry allows users to build custom classes that function as drop-in replacements for core functionality.

For more details on registering custom datasets, see the [tutorial here](https://github.com/oumi-ai/oumi/blob/main/notebooks/Oumi%20-%20Datasets%20Tutorial.ipynb).

For a tutorial on writing a custom cloud/cluster for running jobs, see the [tutorial here](https://github.com/oumi-ai/oumi/blob/main/notebooks/Oumi%20-%20Launching%20Jobs%20on%20Custom%20Clusters.ipynb).

You can find further information about the required registry decorators [here](https://oumi.ai/docs/en/latest/api/oumi.core.registry.html#oumi.core.registry.register_cloud_builder).

# 🧭 What's Next?

Now that you've completed the basic tour, you're ready to tackle the other [notebook guides & tutorials](https://oumi.ai/docs/en/latest/get_started/tutorials.html).

If you have not already, make sure to take a look at the [Quickstart](https://oumi.ai/docs/en/latest/get_started/quickstart.html) for an overview of our CLI.